In [2]:
import platform, os, sys, datetime, re
from os.path import join
from glob import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
# sys.path.append(join(root_dir,'cvtracer'))
from cvt.TrAQ.Trial import Trial
from cvt.TrAQ.Tank import Tank
from cvt.TrAQ.Group import Group
from cvt.TrAQ.CVTracer import CVTracer, create_named_window, wait_on_named_window

In [20]:
trial_files = sorted(glob('../tracking/*/trial.pik'))
display(trial_files)

['../tracking/Pa_Fri_7dpf_GroupA_n2_2020-06-05-120920-0000/trial.pik',
 '../tracking/Pa_Fri_7dpf_GroupA_n2b_2020-06-05-103456-0000/trial.pik',
 '../tracking/Pa_Fri_7dpf_GroupA_n5_2020-06-05-083453-0000/trial.pik',
 '../tracking/Pa_Fri_7dpf_GroupB_n2b_2020-06-05-114635-0000/trial.pik',
 '../tracking/Pa_Fri_7dpf_GroupB_n5_2020-06-05-094643-0000/trial.pik']

In [18]:
val_name  = ['dwall', 'speed', 'omega']
val_range = [ [0, trial.tank.r_cm], 
              [0, trial.tank.r_cm], 
              [-20, 20] ]
val_bins  = [ 40, 40, 40 ]
n_buffer_frames = 2


for trial_file in trial_files:

    trial = Trial()
    trial.load(trial_file)
    trial.convert_pixels_to_cm()
    trial.calculate_kinematics()
    # trial.save()

    tag = trial.evaluate_cuts(n_buffer_frames = n_buffer_frames,
                              ocut = 0,
                              vcut = val_range[1],
                              wcut = val_range[2] )

    trial.calculate_statistics(val_name = val_name, 
                               val_range = val_range, 
                               val_bins = val_bins,
                               ocut = True, vcut = True, wcut = True,
                               tag = tag)
    
    # Plot the joint distribution of pair distance and pair angle.
    trial.group.collect_distance_alignment(ocut = True, 
                                           vcut = True, 
                                           wcut = True)
    dist_align = np.copy(trial.group.dij_mij)
    dist_align[:,1] = 180/np.pi*np.arccos(dist_align[:,1]) # convert to degrees
    plt.hist2d(dist_align[:,0], dist_align[:,1], bins = [20,20], 
               range=[[0,2*trial.tank.r_cm],[0,180]], cmap=plt.cm.hot_r)
    plt.xlabel("Pair distance (cm)")
    plt.ylabel("Pair angle (deg)")
    plt.colorbar()
    plt.savefig(trial.make_fig_file('pairs',tag),**trial.plot_options)
    plt.clf()
    
    # Plot the angle vs time for each fish.
    for f in trial.group.fish:
        t   = f.df['t']
        th  = f.df['theta']
        dth = th.diff()
        dth = dth - 2*np.pi*np.rint(dth/(2*np.pi))
        th  = np.cumsum(dth)
        plt.plot(t,th) #,marker='.')
    plt.xlabel('Time (s)')
    plt.ylabel('Angle (rad)')
    plt.savefig(trial.make_fig_file('angle-vs-time',''),**trial.plot_options)    
    plt.clf()
    


        Trial loaded from ../tracking/Pa_Fri_7dpf_GroupA_n2_2020-06-05-120920-0000/trial.pik 

       Converting pixels to (x,y) space in (cm,cm).

       using to tank size and location from ../tracking/Pa_Fri_7dpf_GroupA_n2_2020-06-05-120920-0000/tank.pik
       Calculating kinematics...

        Trial object saved as ../tracking/Pa_Fri_7dpf_GroupA_n2_2020-06-05-120920-0000/trial.pik 

       ... kinematics calculated for Trial and saved in
             ../tracking/Pa_Fri_7dpf_GroupA_n2_2020-06-05-120920-0000/trial.pik 


  Generating occlusion cut using min d_nn = 0.00 cm with 2 buffer frames
 Shape before cuts:  (34541, 2)
 Shape after cuts:  (34541, 2)
    ... for fish 0, 
 Shape before cuts:  (34541, 2)
 Shape after cuts:  (34541, 2)
    ... for fish 1, 


  Generating speed cut using range [0.00, 0.48] cm/s with 2 buffer frames
    ... for fish 0, 
    ... for fish 1, 


  Generating angular speed cut using range [-20.00, 20.00] rad/s with 2 buffer frames
    ... for fish 0, 
 

 Shape before cuts:  (38253, 2)
 Shape after cuts:  (37312, 2)
 Shape before cuts:  (38253, 2)
 Shape after cuts:  (37256, 2)

        Trial loaded from ../tracking/Pa_Fri_7dpf_GroupB_n5_2020-06-05-094643-0000/trial.pik 

       Converting pixels to (x,y) space in (cm,cm).

       using to tank size and location from ../tracking/Pa_Fri_7dpf_GroupB_n5_2020-06-05-094643-0000/tank.pik
       Calculating kinematics...

        Trial object saved as ../tracking/Pa_Fri_7dpf_GroupB_n5_2020-06-05-094643-0000/trial.pik 

       ... kinematics calculated for Trial and saved in
             ../tracking/Pa_Fri_7dpf_GroupB_n5_2020-06-05-094643-0000/trial.pik 


  Generating occlusion cut using min d_nn = 0.00 cm with 2 buffer frames
 Shape before cuts:  (37359, 2)
 Shape after cuts:  (37359, 2)
    ... for fish 0, 
 Shape before cuts:  (37359, 2)
 Shape after cuts:  (37359, 2)
    ... for fish 1, 
 Shape before cuts:  (37359, 2)
 Shape after cuts:  (37359, 2)
    ... for fish 2, 
 Shape before cuts

<Figure size 432x288 with 0 Axes>